# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.40.0


## Dataset

### Overview
TODO: The dataset 

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
 
from azureml.core import Workspace
from azureml.core import Dataset
from azureml.data.datapath import DataPath

ws = Workspace.from_config()
datastore = ws.get_default_datastore()



In [3]:

from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd

url_path = "https://raw.githubusercontent.com/Vzard/ML-Azure-Capstone/master/StrokeData_V1.csv"
 

In [4]:
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

key = "strokedata_1b"
description_text = "stroke"


In [5]:
dataset = ds.register(workspace=ws,
                                   name=key,
                                   description=description_text)

In [6]:
 

project_folder = './pipeline-project1'

# choose a name for experiment
experiment_name = "Predictstroke_2"

experiment=Experiment(ws, experiment_name)

In [7]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "Capstone"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name="ClusterKal")
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)


Found existing cluster, use it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'AUC_weighted',
    "n_cross_validations": 10
}
 
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="stroke",  
                             enable_early_stopping= True,
                             test_size =0.2,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )


In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config,show_output=True)

Submitting remote run.
No run_configuration provided, running on ClusterKal with default configuration
Running on remote compute: ClusterKal


Experiment,Id,Type,Status,Details Page,Docs Page
Predictstroke_1,AutoML_f5ce51e2-e0f7-44a3-a639-235da7762a96,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the t

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [11]:
print("Run Status: ",remote_run.get_status())
#Must indicate complete to indicate run has completed

Run Status:  Completed


In [12]:
 
for child_run in remote_run.get_children():
    print(child_run,"\n********************\n")
    

Run(Experiment: Predictstroke_1,
Id: AutoML_f5ce51e2-e0f7-44a3-a639-235da7762a96_50,
Type: azureml.scriptrun,
Status: Completed) 
********************

Run(Experiment: Predictstroke_1,
Id: AutoML_f5ce51e2-e0f7-44a3-a639-235da7762a96_51,
Type: azureml.scriptrun,
Status: Completed) 
********************

Run(Experiment: Predictstroke_1,
Id: AutoML_f5ce51e2-e0f7-44a3-a639-235da7762a96_48,
Type: azureml.scriptrun,
Status: Canceled) 
********************

Run(Experiment: Predictstroke_1,
Id: AutoML_f5ce51e2-e0f7-44a3-a639-235da7762a96_49,
Type: azureml.scriptrun,
Status: Canceled) 
********************

Run(Experiment: Predictstroke_1,
Id: AutoML_f5ce51e2-e0f7-44a3-a639-235da7762a96_47,
Type: azureml.scriptrun,
Status: Canceled) 
********************

Run(Experiment: Predictstroke_1,
Id: AutoML_f5ce51e2-e0f7-44a3-a639-235da7762a96_45,
Type: azureml.scriptrun,
Status: Completed) 
********************

Run(Experiment: Predictstroke_1,
Id: AutoML_f5ce51e2-e0f7-44a3-a639-235da7762a96_44,
Type: 

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [151]:
best_child_run,fitted_model = remote_run.get_output()
 


Package:azureml-automl-runtime, training version:1.44.0, current version:1.40.0
Package:azureml-core, training version:1.44.0, current version:1.40.0
Package:azureml-dataprep, training version:4.2.2, current version:3.0.0
Package:azureml-dataprep-rslex, training version:2.8.1, current version:2.4.0
Package:azureml-dataset-runtime, training version:1.44.0, current version:1.40.0
Package:azureml-defaults, training version:1.44.0, current version:1.40.0
Package:azureml-inference-server-http, training version:0.7.4, current version:0.4.11
Package:azureml-interpret, training version:1.44.0, current version:1.40.0
Package:azureml-mlflow, training version:1.44.0, current version:1.40.0
Package:azureml-pipeline-core, training version:1.44.0, current version:1.40.0
Package:azureml-responsibleai, training version:1.44.0, current version:1.40.0
Package:azureml-telemetry, training version:1.44.0, current version:1.40.0
Package:azureml-train-automl-client, training version:1.44.0, current version:1

In [153]:
best_run_metrics=best_child_run.get_metrics()


In [154]:
best_child_run

Experiment,Id,Type,Status,Details Page,Docs Page
Predictstroke_1,AutoML_f5ce51e2-e0f7-44a3-a639-235da7762a96_50,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [155]:
fitted_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('34', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=True)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7922222222222222, learning_rate=0.03158578947368421, max_bin=210, max_depth=3, min_child_weight=2, min_data_in_leaf=0.003457931034482759, min_split_gain=0.894736842105263, n_estimators=600, n_jobs=1, num_leaves=239, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), r

In [157]:
os.makedirs('./outputs',exist_ok=True)

In [169]:
#TODO: Save the best model
import joblib
joblib.dump(fitted_model,filename= "outputs/automl.joblib")

['outputs/automl.joblib']

In [170]:
modelname =  best_child_run.properties['model_name']
modelname

'AutoMLf5ce51e2e50'

In [171]:
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/nanokal/code/Users/vkalakk1/Assignment3


In [172]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('34', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=True)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7922222222222222, learning_rate=0.03158578947368421, max_bin=210, max_depth=3, min_child_weight=2, min_data_in_leaf=0.003457931034482759, min_split_gain=0.894736842105263, n_estimators=600, n_jobs=1, num_leaves=239, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), r

In [163]:
from azureml.core.model import Model
reg_model = remote_run.register_model(model_name = modelname, description="strokepredict_new")

In [178]:
from azureml.automl.core.shared import constants
env = best_child_run.get_environment()
script_file = "./outputs/score.py"

best_child_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)
best_child_run.download_file(constants.CONDA_ENV_FILE_PATH, 'env.yml')


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [179]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.automl.core.shared import constants

In [180]:
servicename = 'azstrokepredict'
print(servicename)

azstrokepredict


In [181]:
#Create Inference Config
inference_config = InferenceConfig(entry_script = script_file, environment=env)


In [182]:

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               enable_app_insights = True,
                                               description ="PredictStroke")

In [183]:

aci_service_name = "predictstroke6"
#deploy the model as a webservice
from azureml.core.webservice import LocalWebservice, Webservice

#set deployment configuration

aci_service = Model.deploy(ws, aci_service_name, [reg_model], inference_config,aciconfig)
aci_service.wait_for_deployment(True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-08-17 20:20:06+00:00 Creating Container Registry if not exists.
2022-08-17 20:20:06+00:00 Registering the environment.
2022-08-17 20:20:07+00:00 Use the existing image.
2022-08-17 20:20:07+00:00 Submitting deployment to compute.
2022-08-17 20:20:12+00:00 Checking the status of deployment predictstroke6..
2022-08-17 20:22:34+00:00 Checking the status of inference endpoint predictstroke6.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [184]:
print('Service state: ' + aci_service.state)
print('Service scoring URI: ' + aci_service.scoring_uri)
print('Service Swagger URI: ' + aci_service.swagger_uri)
 

Service state: Healthy
Service scoring URI: http://125cbf0f-8ecb-4f06-9714-6e91735bf099.centralus.azurecontainer.io/score
Service Swagger URI: http://125cbf0f-8ecb-4f06-9714-6e91735bf099.centralus.azurecontainer.io/swagger.json


In [185]:
scoring_uri = aci_service.scoring_uri
print(scoring_uri)

http://125cbf0f-8ecb-4f06-9714-6e91735bf099.centralus.azurecontainer.io/score


#Test by sending request and view response below

In [191]:

import requests
import json

 
data = {"data":
        [
          {  
            "gender" : "Male",
            "age"   :  80,
            "hypertension" : 1,
            "heart_disease":1,
            "ever_married" :"Yes",
            "work_type": "Govt_job",
            "Residence_type" : "Urban",
            "avg_glucose_level" : 221.29,
            "bmi": 36.6,
            "smoking_status": "smokes"
          },
          {
            "gender" : "Male",
            "age"   :  90,
            "hypertension" : 1,
            "heart_disease": 1,
            "ever_married"  : "Yes",
            "work_type" : "Private",
            "Residence_type" : "Urban",
            "avg_glucose_level" : 228.69,
            "bmi" : 40,
            "smoking_status" : "smokes"           
         },
         {
            "gender" : "Male",
            "age"   :  84,
            "hypertension" : 1,
            "heart_disease": 1,
            "ever_married"  : "Yes",
            "work_type" : "Private",
            "Residence_type" : "Urban",
            "avg_glucose_level" : 228.69,
            "bmi" : 40,
            "smoking_status" : "smokes"          
         }  
      ]
    }
    
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print("prediction is :" , resp.json())

prediction is : {"result": [0, 0, 0]}


In [187]:
#Not deleted the resources, since I have other work on running on it

**Submission Checklist**
- I have registered the model. 
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
